# Segment a Wav Dataset into Frames

## GTzan Dataset

http://marsyasweb.appspot.com/download/data_sets/

The dataset consists of 1000 audio tracks each 30 seconds long.

It contains 10 genres, each represented by 100 tracks.

The tracks are all 22050Hz Mono 16-bit audio files in .wav format.


##### 8 May 2019
#### Alessandro L. Koerich

In [1]:
import numpy as np
import os
import soundfile as sf
import math

import matplotlib.pyplot as plt
import pylab as py
from scipy import fftpack
from scipy import signal
from pylab import *
import scipy.io.wavfile

In [2]:
# Path to source WAV files
path2 = 'gtzan'

# Path to destination (segmentated WAV files)
path3 = 'gtzan_3f_Win_110250_75'

## Build a list of WAV files

In [3]:
# Build a list of WAV files
file_list = []
i = 0 
for path, subdirs, files in os.walk(path2):
    for name in files:
        file_list.append(os.path.join(path, name))
        i += 1
print("Files processed: " + str(i))

Files processed: 1000


In [4]:
file_list

['gtzan/fold3/hiphop/hiphop.00057.au',
 'gtzan/fold3/hiphop/hiphop.00040.au',
 'gtzan/fold3/hiphop/hiphop.00026.au',
 'gtzan/fold3/hiphop/hiphop.00024.au',
 'gtzan/fold3/hiphop/hiphop.00025.au',
 'gtzan/fold3/hiphop/hiphop.00055.au',
 'gtzan/fold3/hiphop/hiphop.00088.au',
 'gtzan/fold3/hiphop/hiphop.00047.au',
 'gtzan/fold3/hiphop/hiphop.00045.au',
 'gtzan/fold3/hiphop/hiphop.00022.au',
 'gtzan/fold3/hiphop/hiphop.00052.au',
 'gtzan/fold3/hiphop/hiphop.00082.au',
 'gtzan/fold3/hiphop/hiphop.00075.au',
 'gtzan/fold3/hiphop/hiphop.00095.au',
 'gtzan/fold3/hiphop/hiphop.00012.au',
 'gtzan/fold3/hiphop/hiphop.00013.au',
 'gtzan/fold3/hiphop/hiphop.00094.au',
 'gtzan/fold3/hiphop/hiphop.00004.au',
 'gtzan/fold3/hiphop/hiphop.00081.au',
 'gtzan/fold3/hiphop/hiphop.00076.au',
 'gtzan/fold3/hiphop/hiphop.00059.au',
 'gtzan/fold3/hiphop/hiphop.00060.au',
 'gtzan/fold3/hiphop/hiphop.00028.au',
 'gtzan/fold3/hiphop/hiphop.00077.au',
 'gtzan/fold3/hiphop/hiphop.00080.au',
 'gtzan/fold3/hiphop/hiph

## Check if the audio samples have at least 5 seconds

In [5]:
# This block of code is to make sure that all of the audio samples have at least XX seconds

# For a sample rate = 22,050 
#  1s = 22,050 x 1  =  22,050 samples 
#  5s = 22,050 x 5  = 110,250 samples
# 10s = 22,050 x 10 = 220,500 samples

length = list()
i      = 0
avg    = 0

for file in file_list:
    try:
        data, samplerate = sf.read(file)
        if len(data) <= 110250:
            print("Audio with less than 5s: "+str(file) )
        #computer average lenght of files
        avg = avg + len(data)
        length.append(len(data))
        i += 1
    except:
        print("Wrong format file skipped.")
    #print(str(samplerate))

print( "Files processed: "+str(i) )

print( "Average file length: "+str(avg/i)+" samples   "+str(avg/i/samplerate)+"s   "+str(avg/i/samplerate/60)+"min" )

print( "Max length: "+str(max(length))+" samples   "+str(max(length)/samplerate)+"s   "+str(max(length)/samplerate/60)+"min" )

print( "Min length: "+str(min(length))+" samples   "+str(min(length)/samplerate)+"s   "+str(min(length)/samplerate/60)+"min" )

Files processed: 1000
Average file length: 662030.846 samples   30.024074648526078s   0.5004012441421013min
Max length: 675808 samples   30.648888888888887s   0.5108148148148148min
Min length: 660000 samples   29.931972789115648s   0.4988662131519275min


## Sliding Window Setup

In [6]:
# window  = size of window (1,600 - 8,000 - 16,000 - 32,000, etc.)
# overlap = percentage of overlapping (0, 0.25, 0.50, 0.75, etc.)

# sample rate = 22,050
#  1s = 22,050 x 1 =  22,050 samples
#  5s = 22,050 x 5 = 110,250
# 10s = 22,050 x 10 = 220,500

def window(data, window, overlap):
    
    overlap = 1 - overlap
    print ( str( window * overlap) )
    print ( str( len(data) / (window*overlap) ) )

    end = int( ceil( ( (len(data) - window ) / (window*overlap) ) ) )
    print(" End: " + str(end) )
    
    # Make a 2-D array of samples
    windowed = []
    if end == 0:
        end = end + 1
    for i in range(0, end):
        
            if len( data[i*( int( window*overlap ) ):i*( int( window*overlap) ) + window] ) == window:
                windowed.append(data[i*( int( window*overlap ) ):i * ( int( window*overlap ) ) + window ] )
                
            if len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])<window:
                spch = np.zeros(window)
                spch[0:len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])]=data[0:len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])]
                spch[len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])+1:window]=data[len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])-1]
                windowed.append(spch)
                print("Warning: shorter than frame size-> zero padding")
                
    return (windowed)

## Energy of the Audio Signal

In [7]:
# Compute the signal energy
def energy(samples):
    return np.sum(np.power(samples, 2.)) / float(len(samples))

## Write Metadata in a CSV File

In [8]:
# WRITE TO FILE METADATA

import csv

filename="gtzan_3f_Win_110250_75.csv"

fl = open(filename, 'w')

writer = csv.writer(fl)

values = ['track_id','frame_id','fold','genre']

writer.writerow(values)

30

## Segment Audio Files with the Sliding Window

### Each segment is saved on disk with the appropriate prefix indicating the segment number (_segment number).

### Segment of low energy (possibly silence) are discarded

In [9]:
# Audio files are segmented and each segment is saved on disk with the approporiate prefix indicating the segment
# number.
# np.boxcar(frame_size)
# sample rate = 22,050
#  1s = 22,050 x 1 =  22,050
#  5s = 22,050 x 5 = 110,250
# 10s = 22,050 x 10 = 220,500

frame_size  = 110250
frame_over  = 0.75
track_id    = 1

for file in file_list:
    try:
        data , samplerate = sf.read( file )
    except:
        continue
    
    print ("--------------")
    print ("Sample Rate: "+str(samplerate)+"   Length: "+str(data.shape))
    print (str( file ))
        
    frame_id = 1
            
    windowed = window(data, frame_size, frame_over)

    for i in range(len( windowed ) ):
        
        spch = windowed[i] * signal.boxcar(frame_size)
        
        ##### Evaluate Energy
        ##### Only save a segment if it has enough energy
        if energy(spch) > 1e-5:
        
            seg_name_new = str(track_id)+'_'+str(frame_id)
            fold_id = file.split('/')[1]
            genre_id = file.split('/')[2]
            directory = path3+'/'+fold_id+'/'+genre_id
            print(directory)
            
            #Verify if it exists, otherwise create it
            if not os.path.exists(directory):
                os.makedirs(directory)
            
            print(directory+'/'+seg_name_new+".wav")
            sf.write( directory+'/'+seg_name_new+".wav", spch, samplerate)
        
            #Write a line in .CSV file
            values = [str(track_id), str(frame_id), str(fold_id), str(genre_id) ]  
        
            frame_id += 1
        
            writer.writerow(values)
            
        else:
            print(file, "Energy = "+str(energy(spch)*100)+"%   - Silence file: "+str(i) )
        
    track_id += 1
        

--------------
Sample Rate: 22050   Length: (661794,)
gtzan/fold3/hiphop/hiphop.00057.au
27562.5
24.010666666666665
 End: 21
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_1.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_2.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_3.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_4.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_5.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_6.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_7.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_8.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_9.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_10.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/1_

gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/5_18.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/5_19.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/5_20.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/5_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
gtzan/fold3/hiphop/hiphop.00055.au
27562.5
24.010666666666665
 End: 21
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_1.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_2.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_3.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_4.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_5.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/6_6.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop

gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_10.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_11.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_12.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_13.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_14.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_15.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_16.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_17.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_18.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_19.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_20.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/10_21.wav
--------------
S

gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_11.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_12.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_13.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_14.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_15.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_16.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_17.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_18.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_19.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_20.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/15_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
gtzan/fold3/hiphop/hiphop.00013.au
27562.5
2

gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/20_19.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/20_20.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/20_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
gtzan/fold3/hiphop/hiphop.00059.au
27562.5
24.010666666666665
 End: 21
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_1.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_2.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_3.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_4.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_5.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_6.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fold3/hiphop/21_7.wav
gtzan_3f_Win_110250_75/fold3/hiphop
gtzan_3f_Win_110250_75/fol

...

gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_9.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_10.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_11.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_12.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_13.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_14.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_15.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_16.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_17.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_18.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_19.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/977_20.wav
gtzan

gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_11.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_12.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_13.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_14.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_15.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_16.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_17.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_18.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_19.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_20.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/982_21.wav
--------------
Sample Rate: 22050   Length: (661504,)
gtzan/fold1/hiphop/hiphop.00003.a

gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_6.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_7.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_8.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_9.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_10.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_11.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_12.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_13.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_14.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_15.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_16.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/987_17.wav
gtzan_3f

gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_13.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_14.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_15.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_16.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_17.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_18.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_19.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_20.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/992_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
gtzan/fold1/hiphop/hiphop.00098.au
27562.5
24.010666666666665
 End: 21
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/993_1.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_W

gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_1.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_2.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_3.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_4.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_5.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_6.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_7.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_8.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_9.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_10.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_11.wav
gtzan_3f_Win_110250_75/fold1/hiphop
gtzan_3f_Win_110250_75/fold1/hiphop/997_12.wav
gtzan_3f_Win_

In [10]:
print("Audio segmentation finished")

Audio segmentation finished
